In [1]:
import requests as rq
from bs4 import BeautifulSoup
import pandas as pd 
import numpy as np
import time
import json
import concurrent.futures 
from multiprocessing.dummy import Pool as pool
import re
import random
from lxml import etree

In [2]:
r = rq.get('https://autumnfish.cn/playlist/detail?id=2829883282')
data = r.text
print(data)

{"code":200,"relatedVideos":null,"playlist":{"subscribers":[{"defaultAvatar":true,"province":350000,"authStatus":0,"followed":false,"avatarUrl":"http://p1.music.126.net/RLeBJe4D1ZzUtltxfoKDMg==/109951163250239066.jpg","accountStatus":0,"gender":0,"city":350200,"birthday":-2209017600000,"userId":1853462382,"userType":0,"nickname":"小小邱邱1","signature":"","description":"","detailDescription":"","avatarImgId":109951163250239070,"backgroundImgId":109951162868126480,"backgroundUrl":"http://p1.music.126.net/_f8R60U9mZ42sSNvdPn2sQ==/109951162868126486.jpg","authority":0,"mutual":false,"expertTags":null,"experts":null,"djStatus":0,"vipType":0,"remarkName":null,"authenticationTypes":0,"avatarDetail":null,"avatarImgIdStr":"109951163250239066","backgroundImgIdStr":"109951162868126486","anchor":false,"avatarImgId_str":"109951163250239066"},{"defaultAvatar":true,"province":0,"authStatus":0,"followed":false,"avatarUrl":"http://p1.music.126.net/ma8NC_MpYqC-dK_L81FWXQ==/109951163250233892.jpg","accountS

In [3]:
print(type(data))

<class 'str'>


In [4]:
data_dict = json.loads(data)

In [126]:
data_dict.get('playlist').get('id')

2829883282

In [5]:
data_json = json.dumps(data_dict, indent=3, ensure_ascii=False)

In [7]:
print(data_json)

{
   "code": 200,
   "relatedVideos": null,
   "playlist": {
      "subscribers": [
         {
            "defaultAvatar": true,
            "province": 350000,
            "authStatus": 0,
            "followed": false,
            "avatarUrl": "http://p1.music.126.net/RLeBJe4D1ZzUtltxfoKDMg==/109951163250239066.jpg",
            "accountStatus": 0,
            "gender": 0,
            "city": 350200,
            "birthday": -2209017600000,
            "userId": 1853462382,
            "userType": 0,
            "nickname": "小小邱邱1",
            "signature": "",
            "description": "",
            "detailDescription": "",
            "avatarImgId": 109951163250239070,
            "backgroundImgId": 109951162868126480,
            "backgroundUrl": "http://p1.music.126.net/_f8R60U9mZ42sSNvdPn2sQ==/109951162868126486.jpg",
            "authority": 0,
            "mutual": false,
            "expertTags": null,
            "experts": null,
            "djStatus": 0,
            "vi

In [11]:
filename = 'test_data.json'
with open(filename, 'w', encoding='utf-8') as fw:
    json.dump(data_dict, fw, ensure_ascii=False)

In [16]:
# 获取所有的歌曲id
track_ids = data_dict.get("playlist").get("trackIds")

In [18]:
print(len(track_ids))

35


In [20]:
song_ids = []
for track in track_ids:
    song_id = track.get('id')
    song_ids.append(song_id)

In [21]:
# 歌单中所有歌曲的id
song_ids

[1372552573,
 1371167139,
 1358935915,
 1445556953,
 1486086007,
 1293951677,
 185700,
 1501704986,
 1382592790,
 30431364,
 450853439,
 492390949,
 1363883359,
 254485,
 461080452,
 176053,
 1430287528,
 1488288505,
 470057375,
 1335580481,
 31654343,
 293948,
 254359,
 1321631043,
 1382592787,
 1408220694,
 489302317,
 1344814341,
 351271,
 1357816405,
 446627373,
 502043537,
 569200210,
 1374086946,
 414706021]

In [24]:
# 歌单的标签列表
tags_list = data_dict.get("playlist").get("tags")

In [25]:
# 歌单名称
name = data_dict.get("playlist").get("name")

In [27]:
# 歌单订阅数量
subscribedCount = data_dict.get("playlist").get("subscribedCount")

In [28]:
# 歌单播放数量
playCount = data_dict.get("playlist").get("playCount")

In [53]:
playlist_info_dict = {
    "playlist_id": 2829883282,
    "song_ids":song_ids,
    "tags_list": tags_list,
    "name": name,
    "subscribedCount": subscribedCount,
    "playCount": playCount,
}

In [54]:
playlist_info_dict

{'playlist_id': 2829883282,
 'song_ids': [1372552573,
  1371167139,
  1358935915,
  1445556953,
  1486086007,
  1293951677,
  185700,
  1501704986,
  1382592790,
  30431364,
  450853439,
  492390949,
  1363883359,
  254485,
  461080452,
  176053,
  1430287528,
  1488288505,
  470057375,
  1335580481,
  31654343,
  293948,
  254359,
  1321631043,
  1382592787,
  1408220694,
  489302317,
  1344814341,
  351271,
  1357816405,
  446627373,
  502043537,
  569200210,
  1374086946,
  414706021],
 'tags_list': ['华语'],
 'name': '[华语私人订制] 最懂你的华语推荐 每日更新35首',
 'subscribedCount': 3933265,
 'playCount': 537350080}

# MongoDB存储测试

In [29]:
import pymongo

In [30]:
myclient = pymongo.MongoClient("mongodb://192.168.206.100:27017/")

In [31]:
# 所有数据库列表
dblist = myclient.list_database_names()

In [32]:
dblist

['admin', 'local', 'recommender', 'springTest']

In [33]:
springTest_db = myclient["springTest"]

In [35]:
springTest_db.collection_names()

E:\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: collection_names is deprecated. Use list_collection_names instead.
  """Entry point for launching an IPython kernel.


['user', 'books']

In [36]:
# 创建集合，如果集合中没有文档则不存在
mycol = springTest_db["test_play_list"]

In [42]:
mydict1 = { "name": "RUN", "alexa": "10000", "url": "https://www.runoob.com" }

In [43]:
# 插入文档，
x = mycol.insert_one(mydict1)

In [44]:
print(x.inserted_id)

5fed2eed21fa8a3ef1a6bc39


In [45]:
mydict2 = { "id": "1", "name": "RUN", "alexa": "10000", "url": "https://www.runoob.com" }

In [46]:
x = mycol.insert_one(mydict2)

In [47]:
print(x.inserted_id)

5fed2fc321fa8a3ef1a6bc3a


In [48]:
# 指定插入的id
mydict3 = { "_id": "1", "name": "RUN", "alexa": "10000", "url": "https://www.runoob.com" }

In [49]:
x = mycol.insert_one(mydict3)

In [50]:
print(x.inserted_id)

1


In [52]:
for x in mycol.find():
    print(x)

{'_id': ObjectId('5fed2eb021fa8a3ef1a6bc38'), 'name': 'RUNOOB', 'alexa': '10000', 'url': 'https://www.runoob.com'}
{'_id': ObjectId('5fed2eed21fa8a3ef1a6bc39'), 'name': 'RUN', 'alexa': '10000', 'url': 'https://www.runoob.com'}
{'_id': ObjectId('5fed2fc321fa8a3ef1a6bc3a'), 'id': '1', 'name': 'RUN', 'alexa': '10000', 'url': 'https://www.runoob.com'}
{'_id': '1', 'name': 'RUN', 'alexa': '10000', 'url': 'https://www.runoob.com'}


In [55]:
mycol.insert_one(playlist_info_dict)

In [56]:
for x in mycol.find():
    print(x)

{'_id': ObjectId('5fed2eb021fa8a3ef1a6bc38'), 'name': 'RUNOOB', 'alexa': '10000', 'url': 'https://www.runoob.com'}
{'_id': ObjectId('5fed2eed21fa8a3ef1a6bc39'), 'name': 'RUN', 'alexa': '10000', 'url': 'https://www.runoob.com'}
{'_id': ObjectId('5fed2fc321fa8a3ef1a6bc3a'), 'id': '1', 'name': 'RUN', 'alexa': '10000', 'url': 'https://www.runoob.com'}
{'_id': '1', 'name': 'RUN', 'alexa': '10000', 'url': 'https://www.runoob.com'}
{'_id': ObjectId('5fed328921fa8a3ef1a6bc3b'), 'playlist_id': 2829883282, 'song_ids': [1372552573, 1371167139, 1358935915, 1445556953, 1486086007, 1293951677, 185700, 1501704986, 1382592790, 30431364, 450853439, 492390949, 1363883359, 254485, 461080452, 176053, 1430287528, 1488288505, 470057375, 1335580481, 31654343, 293948, 254359, 1321631043, 1382592787, 1408220694, 489302317, 1344814341, 351271, 1357816405, 446627373, 502043537, 569200210, 1374086946, 414706021], 'tags_list': ['华语'], 'name': '[华语私人订制] 最懂你的华语推荐 每日更新35首', 'subscribedCount': 3933265, 'playCount': 53

In [95]:
mydict3 = {}

In [96]:
mydict3 = { "id": "2", "name": "RUN", "alexa": "10000", "url": "https://www.runoob.com" }

In [97]:
mycol.insert_one(mydict3)

In [104]:
mydict3 = { "_id": "3", "name": "RUN", "alexa": "10000", "url": "https://www.runoob.com" }

In [105]:
mycol.insert_one(mydict3)

In [106]:
mydict3["_id"] = "4"

In [107]:
mycol.insert_one(mydict3)

对于同一个python对象，如字典mydict3，如果mydict3没有重新赋值，同时也不指定_id，则出入mongodb会出错，有相同的objectID

# Python多线程测试

In [61]:
import threading
import time

def coding():
    for x in range(3):
        print('%s正在写代码' % x)
        time.sleep(1)

def drawing():
    for x in range(3):
        print('%s正在画图' % x)
        time.sleep(1)
    
    
def single_thread():
    start_time = time.time()
    coding()
    drawing()
    end_time = time.time()
    print("运行时间："+str(end_time-start_time))

def multi_thread():
    start_time = time.time()
    t1 = threading.Thread(target=coding)
    t2 = threading.Thread(target=drawing)
    
    t1.start()
    t2.start()
    end_time = time.time()
    print("运行时间："+str(end_time-start_time))

In [62]:
single_thread()

0正在写代码
1正在写代码
2正在写代码
0正在画图
1正在画图
2正在画图
运行时间：6.0025599002838135


In [64]:
multi_thread()

0正在写代码
0正在画图运行时间：0.009995698928833008

1正在写代码
1正在画图
2正在写代码
2正在画图


## 使用类进行封装

In [65]:
class CodingThread(threading.Thread):
    def run(self):
        for x in range(3):
            print('%s正在写代码' % x)
            time.sleep(1)
            
class DrawingThread(threading.Thread):
    def run(self):
        for x in range(3):
            print('%s正在画图' % x)
            time.sleep(1)

In [71]:
def multi_thread(): 
    t1 = CodingThread()
    t2 = DrawingThread()
    
    t1.start()
    t2.start()

In [72]:
start_time = time.time()
multi_thread()
end_time = time.time()
print("运行时间："+str(end_time-start_time))

0正在写代码
0正在画图
运行时间：0.009994745254516602
1正在写代码
1正在画图
2正在写代码
2正在画图


## 锁的机制

In [92]:
tickets = 0
def get_ticket():
    global tickets
    for x in range(100000):
        tickets += 1
    print('tickets:%d'%tickets)

def main():
    for x in range(3):
        t = threading.Thread(target=get_ticket)
        t.start()

In [93]:
main()

tickets:100000
tickets:248464
tickets:300000


## 简单多线程

In [112]:
import threading
import time

# 为线程定义一个函数
class myThread(threading.Thread):
    def __init__(self, name, delay):
        threading.Thread.__init__(self)
        self.name = name
        self.delay = delay
        
    def run(self):
        print("Starting " + self.name)
        print_time(self.name, self.delay)
        print("Exiting " + self.name)
        
def print_time(threadNmae, delay):
    counter = 0
    while counter < 3:
        time.sleep(delay)
        print(threadNmae, time.ctime())
        counter += 1

In [116]:
threads = []

# 创建新线程
thread1 = myThread(name="Thread-1", delay=1)
thread2 = myThread(name="Thread-2", delay=2)

# 开启新线程
thread1.start()
thread2.start()

# 添加新线程到线程列表
threads.append(thread1)
threads.append(thread2)

# 等待所有线程完成
for t in threads:
    t.join()
    
print("Exiting Main Thread")

Starting Thread-1
Starting Thread-2
Thread-1 Thu Dec 31 11:48:33 2020
Thread-1 Thu Dec 31 11:48:34 2020
Thread-2 Thu Dec 31 11:48:34 2020
Thread-1 Thu Dec 31 11:48:35 2020
Exiting Thread-1
Thread-2 Thu Dec 31 11:48:36 2020
Thread-2 Thu Dec 31 11:48:38 2020
Exiting Thread-2
Exiting Main Thread


In [122]:
a = ['as', 'as', 'ss', 'er', '34']
b = {}

In [123]:
for index, item in enumerate(a):
    print(str(index) + ": "+ item)
    b[index] = item

0: as
1: as
2: ss
3: er
4: 34


In [124]:
print(b)

{0: 'as', 1: 'as', 2: 'ss', 3: 'er', 4: '34'}


In [128]:
from tqdm import tqdm

pbar = tqdm(["a", "b", "c", "d"], leave=True)
for char in pbar:
#     pbar.set_description("Now Processing %s" % char) #这里的这个方法就相当于给tqdm类的构造函数添加一个 desc 参数
    time.sleep(0.2)

100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00,  4.91it/s]
